In [12]:
import numpy as np
import pandas as pd
import torch
import os

ModuleNotFoundError: No module named 'pandas'

In [3]:
import sys

In [11]:
os.environ['VIRTUAL_ENV']

NameError: name 'os' is not defined

In [9]:
sys.prefix

'/public/mick.perl/venv'

In [10]:
sys.base_prefix

'/public/mick.perl/venv'

In [6]:
os.chdir("data/")
os.getcwd()


'c:\\Users\\miche\\Documents\\GitHub\\DataAnalyticsProject\\data'